# Training

In [1]:
%cd '../'

import os
import sys
import random

# sys.path.append('../')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision
import torchvision.transforms as tra
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.transforms import functional as F
import albumentations as A

import agml
import agml.train.utils
from agml.train.engine import train_one_epoch, evaluate, validate
from agml.train.datatools import COCOObjectDetectionDataset, get_box_transform

/Users/jmearles/Library/Mobile Documents/com~apple~CloudDocs/Documents/ucd/projects/AgML


/Users/jmearles/Library/Mobile Documents/com~apple~CloudDocs/Documents/ucd/projects/AgML/agml/helios_config.sh: line 18: [: missing `]'


Installing Helios
Already up to date.


From https://github.com/PlantSimulationLab/Helios
 * branch              HEAD       -> FETCH_HEAD


## Generate the data

In [2]:
from agml.data.syntheticdata import HeliosDataGenerator

In [3]:
# !mkdir ../agml/Helios
# !git clone https://github.com/PlantSimulationLab/Helios.git ../agml/Helios

In [4]:
hdg = HeliosDataGenerator()

In [5]:
hdg.canopy_param_ranges['VSPGrapevine']['grape_subdivisions'] = [[4.0, 4.0]]

In [6]:
hdg.generate_data(n_imgs=1, canopy_type='VSPGrapevine', simulation_type='rgb')

ITERATION 0
-- Symbol prefix: 
-- Building with BZip2
-- Creating directory /Users/jmearles/Library/Mobile Documents/com~apple~CloudDocs/Documents/ucd/projects/AgML/agml/_helios/build/plugins/visualizer/lib/freetype-2.7/include/freetype/config
-- Creating file /Users/jmearles/Library/Mobile Documents/com~apple~CloudDocs/Documents/ucd/projects/AgML/agml/_helios/build/plugins/visualizer/lib/freetype-2.7/include/freetype/config/ftconfig.h


CMake Deprecation Warning at CMakeLists.txt:18 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Deprecation Warning at Helios/core/CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Deprecation Warning at Helios/core/lib/zlib/CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Deprecation Warning at Helios/

-- Creating file /Users/jmearles/Library/Mobile Documents/com~apple~CloudDocs/Documents/ucd/projects/AgML/agml/_helios/build/plugins/visualizer/lib/freetype-2.7/include/freetype/config/ftoption.h
-- Using Cocoa for window creation
-- Configuring done


CMake Deprecation Warning at Helios/plugins/visualizer/lib/libjpeg-9a/CMakeLists.txt:8 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


DIST_NAME: libjpeg
DIST_VERSION: 9.1.0
DIST_LICENSE: jpeg license
DIST_AUTHOR: Tom Lane, Guido Vollbeding, Philip Gladstone, Bill Allombert, Jim Boucher, Lee Crocker, Bob Friesenhahn, Ben Jackson, Julian Minguillon, Luis Ortiz, George Phillips, Davide Rossi, Ge
DIST_MAINTAINER: Peter Kapec
DIST_URL: http://www.ijg.org/
DIST_DESC: Independent JPEG Group
DIST_DEPENDS: 
-- loading plug-in canopygenerator
CMake Deprecation Warning at Helios/plugins/canopygenerator/CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...

-- Generating done
-- Build files have been written to: /Users/jmearles/Library/Mobile Documents/com~apple~CloudDocs/Documents/ucd/projects/AgML/agml/_helios/build
[  3%] Built target canopygenerator
[ 26%] Built target jpeg
[ 32%] Built target genfiles
[ 41%] Built target zlibstatic
[ 52%] Built target png_static
[ 54%] Built target helios
[ 62%] Built target glfw
[ 85%] Built target freetype
[ 86%] Built target glew_s
[ 87%] Built target visualizer
Scanning dependencies of target executable_exe
[ 87%] Building CXX object CMakeFiles/executable_exe.dir/main.o
[ 88%] Linking CXX executable executable_exe
[ 88%] Built target executable_exe
[ 96%] Built target zlib
[ 97%] Built target glew
[ 98%] Built target visualinfo
[100%] Built target glewinfo


make[1]: Circular lib/all <- lib/all dependency dropped.
make[1]: Circular lib/all <- lib/all dependency dropped.


Reading XML file: ../xmloutput_for_helios/tmp_canopy_params_image.xml...Building canopy of VSP grapevine...done.
Canopy consists of 3321 leaves and 526496 total primitives.
Ground geometry...done.
Ground consists of 100 total primitives.
done.
Elapsed time is 1.71969 seconds: Time to generate geometry
camera perspective # 0
Opening graphic window...done.
Updating the plot...Adding 526.596K Context primitives to visualizer....done.
done.
writing JPEG image: ../output/images/RGB_rendering_0_0.jpeg
Elapsed time is 30.2747 seconds: Time to render and write RGB image
Opening graphic window...done.
Updating the plot...Adding 526.596K Context primitives to visualizer....done.
done.
Elapsed time is 29.6201 seconds: Time to render and write full labeled image
129 unique elements out of 3998000
Updating the plot...Adding 2.112K Context primitives to visualizer....done.
done.
Elapsed time is 0.094063 seconds: render
Elapsed time is 0.745922 seconds: image write
Updating the plot...Adding 2.4K Con

KeyboardInterrupt: 

## Load up the model

In [ ]:

#define the number of class
num_classes=2 #background and grape
# load an instance segmentation model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


## Create the Dataset Object and dataloader

In [ ]:
import torchvision.datasets as dset

# the dataset object
img_path="./images"
json_path="./trainval_detection.json"

from datatools import COCOObjectDetectionDataset, get_box_transform


#you can add your own albumentations transforms here 

transform_list = A.Compose([
        #A.Resize(200, 300),
        #A.CenterCrop(100, 100),
        #A.RandomCrop(80, 80),
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=(-6, 6), p=0.6),
        #A.ShiftScaleRotate(rotate_limit=50, p=0.6)
        #A.SmallestMaxSize(max_size=1292, p=1)
        #A.augmentations.transforms.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.15, rotate_limit=5, p=0.5)
        A.RandomScale(scale_limit=0.2, p=0.5)
        #A.core.composition.OneOf([A.Resize(2298, 1292, interpolation=1, always_apply=False, p=1), A.Resize(3109, 1748, interpolation=1, always_apply=False, p=1), A.Resize(2568, 1444, interpolation=1, always_apply=False, p=1), A.Resize(2839, 1596, interpolation=1, always_apply=False, p=1)],p=0.5)
        #A.augmentations.transforms.Resize(2298, 1292, interpolation=1, always_apply=False, p=1)
        #A.VerticalFlip(p=0.5),
        #A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


trainval = COCOObjectDetectionDataset(root = img_path,json_dir = json_path, transforms=transform_list, train=True)


# trainval = dset.CocoDetection(root = img_path,annFile = json_path)#dataset_test = GrapeBunchDataset('test', get_transform(train=False))

train, val = torch.utils.data.random_split(trainval, [2, 1]) # put 2 images in train and 1 in val for this dummy example



# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True, num_workers=0, collate_fn=utils.collate_fn)

#data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=4, collate_fn=utils.collate_fn)

data_loader_validate = torch.utils.data.DataLoader(train, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)
    

## Now lets train for 50 epochs

In [ ]:
# now start the training process
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.009, momentum=0.9, weight_decay=0.0006)

# and a learning rate scheduler (optional)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=1)  # optional lr scheudler (gamma is 1 by default which means no scheduling)

# let's train it for 50 epochs and save after best epoch
num_epochs = 50

best_val_loss=999
for epoch in range(num_epochs):
    model.train()
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=2)
    # update the learning rate (optional)
    lr_scheduler.step()
    # evaluate and checkpoint using the validate dataset
    best_val_loss=validate(model, data_loader_validate, device=device, best_val_loss=best_val_loss)


print('Trained for '+str(num_epochs) + ' epochs')